In [1]:
import os
from covidanalytics import etl as ce
from covidanalytics import analytics as ca
from covidanalytics import plots as cp

In [156]:
import importlib
importlib.reload(cp)

<module 'covidanalytics.plots' from '/Users/michaeln/Documents/code/covid-analytics/covidanalytics/plots.py'>

In [2]:
reports_path=os.path.join(os.environ['HOME'],'Documents/code/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/')
html_path=os.path.join(os.environ['HOME'],'Documents/code/covid-static/pages/')

In [148]:
param_dict = {'reports_path': reports_path,
          'html': {'dir': html_path + 'usa.html',
                   'img': 'usa.png',
                   'img_shape': 'wide'
                  },
          'state': 'California',
          'start_date': '01/22/2020',
          'end_date': '04/01/2020',
          'title': 'California - COVID-19'
         }

In [149]:
filenames = ce.dates_to_filenames(param_dict['start_date'], param_dict['end_date'])

In [150]:
recs = []
for f in filenames:
    try:
        f_path = param_dict['reports_path'] + f
        covid = ce.covid_csv_to_df(f_path)
        if 'state' in param_dict.keys():
            daily = ce.covid_province_state(covid, param_dict['state'])
        if 'country' in param_dict.keys():
            daily = ce.covid_country_region(covid, param_dict['country'])
        if 'query' in param_dict.keys():
            daily = ce.covid_query(covid, param_dict['query'])
        recs.append(daily)
    except:
        print(f)


In [151]:
covid_ts = ca.covid_dict_to_timeseries_df(recs, param_dict['start_date'], param_dict['end_date'])

In [152]:
covid_ts['7day_fit_m'] = np.zeros((len(covid_ts.index), 1))

In [153]:
for i in range(0,(covid_ts.shape[0]-7)):
    if i==0:
        mb = ca.daily_lin_fit(7,covid_ts['confirmed'][-7::].values)
        covid_ts.at[covid_ts.index[-1],'7day_fit_m']=mb['m']
    else:
        mb = ca.daily_lin_fit(7,covid_ts['confirmed'][((i*-1)-7):(i*-1)].values)
        covid_ts.at[covid_ts.index[((i+1)*-1)],'7day_fit_m']=mb['m']

In [154]:
covid_ts.tail(10)

,province_state,confirmed,deaths,active,date,total_change,percent_change,7day_fit_m
2020-03-23,California,2108.0,39.0,2069.0,2020-03-23,462.0,28.068044,229.714286
2020-03-24,California,2538.0,50.0,2488.0,2020-03-24,430.0,20.398482,290.785714
2020-03-25,California,2998.0,65.0,2933.0,2020-03-25,460.0,18.124507,343.000000
2020-03-26,California,3899.0,81.0,3818.0,2020-03-26,901.0,30.053369,440.214286
2020-03-27,California,4657.0,94.0,4563.0,2020-03-27,758.0,19.440882,545.535714
2020-03-28,California,5095.0,110.0,4985.0,2020-03-28,438.0,9.405196,600.214286
2020-03-29,California,5852.0,124.0,5728.0,2020-03-29,757.0,14.857704,643.035714
2020-03-30,California,7138.0,146.0,6992.0,2020-03-30,1286.0,21.975393,739.428571
2020-03-31,California,8210.0,173.0,8037.0,2020-03-31,1072.0,15.018212,832.464286
2020-04-01,California,9855.0,215.0,9640.0,2020-04-02,1645.0,20.036541,964.892857


In [157]:
plt=cp.plot_total_change(covid_ts)

Loading BokehJS ...